In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import uvicorn

app = FastAPI(title="MNG Kargo Mock API", version="1.0.0")

# Request Models - Tek düz yapı
class NearestBranchRequest(BaseModel):
    latitude: float
    longitude: float
    city: str
    district: str
    radius: Optional[int] = 5
    max_results: Optional[int] = 10

class ShipmentRequest(BaseModel):
    sender_name: str
    sender_phone: str
    sender_email: str
    sender_address: str
    sender_city: str
    sender_district: str
    sender_postal_code: str
    receiver_name: str
    receiver_phone: str
    receiver_email: str
    receiver_address: str
    receiver_city: str
    receiver_district: str
    receiver_postal_code: str
    package_description: str
    package_weight: float
    package_length: int
    package_width: int
    package_height: int
    package_value: float
    service_type: str  # standard, express, cargo
    payment_method: str  # sender_pay, receiver_pay
    insurance: Optional[bool] = False
    sms_notification: Optional[bool] = True

# Response Models
class Branch(BaseModel):
    branch_id: str
    branch_name: str
    address: str
    phone: str
    distance: float
    working_hours: str
    latitude: float
    longitude: float

class NearestBranchResponse(BaseModel):
    success: bool
    branches: List[Branch]

class ShipmentResponse(BaseModel):
    success: bool
    shipment_id: str
    tracking_number: str
    barcode: str
    estimated_delivery: str
    shipping_cost: float
    total_cost: float
    service_name: str

class TrackingEvent(BaseModel):
    date: str
    status: str
    location: str
    description: str

class TrackingResponse(BaseModel):
    success: bool
    tracking_number: str
    status: str
    current_location: str
    estimated_delivery: str
    events: List[TrackingEvent]

# Mock Data
MOCK_BRANCHES = [
    Branch(
        branch_id="IST001",
        branch_name="MNG Kargo Kadıköy Şubesi",
        address="Atatürk Cad. No:156 Kadıköy/İstanbul",
        phone="0216 555 0123",
        distance=1.2,
        working_hours="Pzt-Cmt: 08:00-19:00, Pazar: Kapalı",
        latitude=40.9897,
        longitude=29.0245
    ),
    Branch(
        branch_id="IST002", 
        branch_name="MNG Kargo Beşiktaş Şubesi",
        address="Barbaros Bulvarı No:78 Beşiktaş/İstanbul",
        phone="0212 555 0456",
        distance=2.8,
        working_hours="Pzt-Cmt: 08:00-19:00, Pazar: Kapalı",
        latitude=41.0422,
        longitude=29.0094
    ),
    Branch(
        branch_id="IST003",
        branch_name="MNG Kargo Üsküdar Şubesi", 
        address="Hakimiyeti Milliye Cad. No:45 Üsküdar/İstanbul",
        phone="0216 555 0789",
        distance=3.5,
        working_hours="Pzt-Cmt: 08:00-19:00, Pazar: Kapalı",
        latitude=41.0276,
        longitude=29.0174
    )
]

MOCK_TRACKING_DATA = {
    "123456789012": TrackingResponse(
        success=True,
        tracking_number="123456789012",
        status="in_transit",
        current_location="Ankara Transfer Merkezi",
        estimated_delivery="2024-05-30",
        events=[
            TrackingEvent(
                date="2024-05-27 14:30",
                status="picked_up",
                location="İstanbul Kadıköy Şubesi",
                description="Gönderi şubeden alındı"
            ),
            TrackingEvent(
                date="2024-05-27 18:45", 
                status="in_transit",
                location="İstanbul Transfer Merkezi",
                description="Gönderi transfer merkezinde"
            ),
            TrackingEvent(
                date="2024-05-28 09:15",
                status="in_transit", 
                location="Ankara Transfer Merkezi",
                description="Gönderi varış şehrine ulaştı"
            )
        ]
    ),
    "987654321098": TrackingResponse(
        success=True,
        tracking_number="987654321098",
        status="delivered",
        current_location="Teslim Edildi",
        estimated_delivery="2024-05-26",
        events=[
            TrackingEvent(
                date="2024-05-25 10:00",
                status="picked_up",
                location="Ankara Çankaya Şubesi",
                description="Gönderi şubeden alındı"
            ),
            TrackingEvent(
                date="2024-05-25 16:30",
                status="in_transit",
                location="Ankara Transfer Merkezi", 
                description="Gönderi transfer merkezinde"
            ),
            TrackingEvent(
                date="2024-05-26 11:45",
                status="out_for_delivery",
                location="İzmir Konak Şubesi",
                description="Gönderi dağıtıma çıktı"
            ),
            TrackingEvent(
                date="2024-05-26 15:20",
                status="delivered",
                location="Alıcı Adresi",
                description="Gönderi teslim edildi - Alıcı: Ayşe D."
            )
        ]
    )
}

# API Endpoints
@app.get("/")
async def root():
    return {"message": "MNG Kargo Mock API", "version": "1.0.0"}

@app.post("/api/v1/branches/nearest", response_model=NearestBranchResponse)
async def get_nearest_branches(request: NearestBranchRequest):
    """En yakın MNG şubelerini bul"""
    
    # Basit mesafe filtresi (mock)
    filtered_branches = [
        branch for branch in MOCK_BRANCHES 
        if branch.distance <= request.radius
    ]
    
    # Maksimum sonuç sayısını uygula
    if request.max_results:
        filtered_branches = filtered_branches[:request.max_results]
    
    return NearestBranchResponse(
        success=True,
        branches=filtered_branches
    )

@app.post("/api/v1/shipments/prepare", response_model=ShipmentResponse)
async def prepare_shipment(request: ShipmentRequest):
    """Yeni gönderi hazırla"""
    
    # Basit validasyon
    if request.package_weight <= 0 or request.package_weight > 30:
        raise HTTPException(status_code=400, detail="Ağırlık 0.1-30 kg arası olmalı")
    
    if not request.sender_phone.startswith("05") or len(request.sender_phone) != 11:
        raise HTTPException(status_code=400, detail="Gönderici telefon numarası 05xxxxxxxxx formatında olmalı")
    
    if not request.receiver_phone.startswith("05") or len(request.receiver_phone) != 11:
        raise HTTPException(status_code=400, detail="Alıcı telefon numarası 05xxxxxxxxx formatında olmalı")
    
    # Mock response
    import random
    shipment_id = f"MNG{random.randint(100000, 999999)}"
    tracking_number = f"{random.randint(100000000000, 999999999999)}"
    
    # Servis tipine göre fiyat hesaplama (mock)
    base_cost = 15.0
    if request.service_type == "express":
        base_cost = 25.0
    elif request.service_type == "cargo":
        base_cost = 35.0
    
    # Ağırlık bazlı ek ücret
    weight_cost = request.package_weight * 2.0
    insurance_cost = request.package_value * 0.02 if request.insurance else 0.0
    total_cost = base_cost + weight_cost + insurance_cost
    
    return ShipmentResponse(
        success=True,
        shipment_id=shipment_id,
        tracking_number=tracking_number,
        barcode=f"MNG{tracking_number}",
        estimated_delivery="2024-05-30",
        shipping_cost=base_cost + weight_cost,
        total_cost=total_cost,
        service_name=f"MNG {request.service_type.title()}"
    )

@app.get("/api/v1/tracking/{tracking_number}", response_model=TrackingResponse)
async def track_shipment(tracking_number: str):
    """Kargo takip et"""
    
    if len(tracking_number) != 12 or not tracking_number.isdigit():
        raise HTTPException(status_code=400, detail="Takip numarası 12 haneli sayı olmalı")
    
    # Mock data'dan ara
    if tracking_number in MOCK_TRACKING_DATA:
        return MOCK_TRACKING_DATA[tracking_number]
    
    # Bulunamadıysa mock response
    return TrackingResponse(
        success=True,
        tracking_number=tracking_number,
        status="not_found",
        current_location="Bilinmiyor",
        estimated_delivery="",
        events=[
            TrackingEvent(
                date="2024-05-27 12:00",
                status="created",
                location="Sistem",
                description="Gönderi kaydı oluşturuldu"
            )
        ]
    )

@app.get("/api/v1/health")
async def health_check():
    """Sağlık kontrolü"""
    return {"status": "healthy", "timestamp": "2024-05-27T12:00:00Z"}

# Error Handlers
@app.exception_handler(HTTPException)
async def http_exception_handler(request, exc):
    return {
        "success": False,
        "error": {
            "code": exc.status_code,
            "message": exc.detail
        }
    }

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)